In [9]:
import pandas as pd
import plotly.express as px

poke = pd.read_csv('pokedex_limpa.csv')

In [10]:
# Agrupando por geração e contando a quantidade de pokemons por geração
poke_generation = poke.groupby('generation').agg({
    'name':'count'
}).reset_index()

# Criando o gráfico
fig = px.bar(poke_generation, x='generation', y='name', title='Total de Pokémon por Geração', color='generation',
            labels={
                'generation':'Geração',
                'name':'Total de Pokémon por Geração'
            })

# Adiconando a linha para seguir o topo das barras
fig.add_scatter(x=poke_generation['generation'], y=poke_generation['name'], 
                 mode='lines+markers', name='Linha Acompanhante', 
                 line=dict(color='red', width=2))

fig.show()

In [11]:
# Agrupando os pkemons pela geração e fazendo a média dos seus atributos
poke_status = poke.groupby('generation').agg({
    'attack':'mean',
    'defense':'mean',
    'speed':'mean'
}).reset_index()

# Criando uma nova coluna tendo a média geral dos pokemons de cada geração
poke_status['mean_total'] = poke_status[['attack', 'defense', 'speed']].mean(axis=1)

# Criando o gráfico
fig = px.bar(poke_status, x='generation',y='mean_total',
            labels={
             'mean_total':'Média geral dos atributos',
             'generation':'Geração'
            })

# Adicionando as linhas das médias
fig.add_scatter(x=poke_status['generation'], y=poke_status['attack'], 
                 mode='lines+markers', name='Média de attack por geração', 
                 line=dict(color='red', width=2))

fig.add_scatter(x=poke_status['generation'], y=poke_status['defense'], 
                 mode='lines+markers', name='Média de defense por geração', 
                 line=dict(color='green', width=2))

fig.add_scatter(x=poke_status['generation'], y=poke_status['speed'], 
                 mode='lines+markers', name='Média de speed por geração', 
                 line=dict(color='purple', width=2))


fig.show()

 Os jogos de pokemons costumam ser balanceados, podemos notar isso na média total dos atributos entre as gerações que não variam muito. Porém podemos notar que existe uma leve tendencia de aumento da média de attack e defence nass últimas gerações.
 Alguns dos possíveis motivos são: O level desing dos pokemons mudam conforme as gerações, quase sempre acompanhando a história do próprio jogo, como a região em que eles estão e o nível das batalhas da região.
 Um detalhe que se deve notar é que na sexta geração ouve um declinio na média de attack e um aumento dos outros atributos. Essa geração é considerada uma das "piores" gerações para os fãs da franquia. Isso pode se dar pelo fato de que o desing dos pokemons serem mais "amigáveis" e menos "Agressivos", detalhe que os fãs não gostaram. porém a franquia tem criado um certo padrão na ditribuição desde então.

## Seleção de Pokémon para a Bolsa:

### Simule a situação na qual você receberá 3 Pokémon de 10 tipos diferentes, selecionados aleatoriamente. Desses 30 Pokémon, você deve automatizar a seleção de 6 Pokémon para a sua bolsa com base em um critério otimizado.

In [12]:
#transforma os tipos únicos em uma lista
unique_types = poke['type_1'].unique().tolist()

#função que pega o dataframe, a coluna tipo e seleciona um pedaço daquele dataframe baseado no parâmetro
def generate_pokemon(df, type, quantity):
  pokemon_type = df[df['type_1'] == type]

  selected_pokemons = pokemon_type.sample(n=quantity)

  return selected_pokemons

selected_pokemons_final = []

for type in unique_types[:10]:
  selected_pokemons = generate_pokemon(poke, type, 3)
  selected_pokemons_final.append(selected_pokemons)

df_result = pd.concat(selected_pokemons_final, ignore_index=True)
df_result = df_result.drop(columns=['Unnamed: 0'])

### Para isso, utilize uma métrica ponderada que leve em consideração os atributos attack, defense, speed e as colunas de against, aplicando pesos customizados para cada critério. Além disso, a equipe final deve conter tipos variados, sem repetição de tipos.

In [13]:
attack = df_result['attack'].tolist()
defense = df_result['defense'].tolist()
speed = df_result['speed'].tolist()

attack_weight = 0.40
defense_weight = 0.30
speed_weight = 0.30

new_attack = [num * attack_weight for num in attack]
new_defense = [num * defense_weight for num in defense]
new_speed = [num * speed_weight for num in speed]

test = df_result
test['pondered_attack'] = new_attack
test['pondered_defense'] = new_defense
test['pondered_speed'] = new_speed
test.describe()

,pokedex_number,generation,type_number,height_m,weight_kg,abilities_number,total_points,hp,attack,defense,...,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy,pondered_attack,pondered_defense,pondered_speed
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,316.233333,3.033333,1.400000,1.140000,41.876667,2.633333,416.033333,64.933333,74.000000,70.666667,...,0.800000,1.216667,0.933333,0.900000,0.950000,1.000000,0.950000,29.600000,21.200000,20.680000
std,251.888547,2.059182,0.498273,0.618452,49.111047,0.490133,83.848995,21.495683,29.192583,27.890591,...,0.406838,0.750670,0.449776,0.305129,0.273861,0.508548,0.422431,11.677033,8.367177,7.515061
min,23.000000,1.000000,1.000000,0.100000,0.100000,2.000000,210.000000,35.000000,30.000000,30.000000,...,0.500000,0.500000,0.000000,0.000000,0.500000,0.500000,0.500000,12.000000,9.000000,10.500000
25%,100.500000,1.000000,1.000000,0.700000,9.600000,2.000000,362.750000,50.000000,57.000000,52.750000,...,0.500000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,22.800000,15.825000,13.875000
50%,281.000000,3.000000,1.000000,1.050000,25.500000,3.000000,439.500000,60.000000,64.000000,70.000000,...,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,25.600000,21.000000,18.750000
75%,553.500000,5.000000,2.000000,1.300000,44.350000,3.000000,484.500000,73.750000,90.000000,83.750000,...,1.000000,1.750000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,25.125000,27.000000
max,748.000000,7.000000,2.000000,2.500000,200.500000,3.000000,510.000000,105.000000,140.000000,152.000000,...,2.000000,4.000000,2.000000,1.000000,2.000000,2.000000,2.000000,56.000000,45.600000,33.600000


In [14]:
test['pondered_values'] = (test['pondered_attack'] + test['pondered_defense'] + test['pondered_speed']) / 100
q3 = test['pondered_values'].quantile(0.75)
test_pondered = test[(test['pondered_values']> q3 )]
test_pondered = test_pondered.drop_duplicates(subset=['type_1'])
choosen_pokemon = test_pondered.head(6)
choosen_pokemon

,pokedex_number,name,generation,status,species,type_number,type_1,type_2,height_m,weight_kg,...,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy,pondered_attack,pondered_defense,pondered_speed,pondered_values
5,555,Darmanitan Standard Mode,5,Normal,Blazing Pokémon,1,Fire,NaN,1.3,92.9,...,2.0,1.0,1.0,1.0,0.5,0.5,56.0,16.5,28.5,1.010
6,99,Kingler,1,Normal,Pincer Pokémon,1,Water,NaN,1.3,60.0,...,1.0,1.0,1.0,1.0,0.5,1.0,52.0,34.5,22.5,1.090
10,545,Scolipede,5,Normal,Megapede Pokémon,2,Bug,Poison,2.5,200.5,...,2.0,1.0,1.0,1.0,1.0,0.5,40.0,26.7,33.6,1.003
13,115,Kangaskhan,1,Normal,Parent Pokémon,1,Normal,NaN,2.2,80.0,...,1.0,0.0,1.0,1.0,1.0,1.0,38.0,24.0,27.0,0.890
16,748,Toxapex,7,Normal,Brutal Star Pokémon,2,Poison,Water,0.7,14.5,...,1.0,1.0,1.0,1.0,0.5,0.5,25.2,45.6,10.5,0.813
18,26,Raichu,1,Normal,Mouse Pokémon,1,Electric,NaN,0.8,30.0,...,1.0,1.0,1.0,1.0,0.5,1.0,36.0,16.5,33.0,0.855


### Crie um gráfico de dispersão 3D para visualizar attack, defense e speed dos Pokémon selecionados.

In [16]:
fig = px.scatter_3d(choosen_pokemon, x='attack', y='defense', z='speed', height=500, title='Dispersão entre Ataque X Velocidade X Defesa dos Pokemons' ,labels={
  'speed': 'Velocidade',
  'attack': 'Ataque',
  'defense': 'Defesa'
})
fig.show()